## 1. Setup - Clone Repository

In [1]:
# Go to /content first, clean previous clone, then clone fresh
%cd /content
!rm -rf Deep_Learning_Project_Gil_Alon
!git clone https://github.com/gil-attar/Deep_Learning_Project_Gil_Alon.git
%cd Deep_Learning_Project_Gil_Alon

/content
Cloning into 'Deep_Learning_Project_Gil_Alon'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 51 (delta 19), reused 34 (delta 9), pack-reused 0 (from 0)
Receiving objects: 100% (51/51), 22.07 KiB | 3.15 MiB/s, done.
Resolving deltas: 100% (19/19), done.
/content/Deep_Learning_Project_Gil_Alon


In [2]:
# Install dependencies
!pip install -r requirements.txt -q

## 2. Set API Keys

Replace with your actual keys below, or use Colab Secrets (🔑 icon in sidebar)

In [3]:
import os

# Option A: Set directly (replace with your keys)
os.environ["ROBOFLOW_API_KEY"] = "zEF9icmDY2oTcPkaDcQY"

# Option B: Use Colab Secrets (recommended)
# from google.colab import userdata
# os.environ["ROBOFLOW_API_KEY"] = userdata.get('ROBOFLOW_API_KEY')

## 3. Download Dataset

In [4]:
!python scripts/download_dataset.py

loading Roboflow workspace...
loading Roboflow project...
Trying version 2...
Version 2 not found, trying next...
Trying version 1...
Exporting format yolov8 in progress : 85.0%
Version export complete for yolov8 format
Successfully downloaded version 1

Dataset downloaded to: /content/Deep_Learning_Project_Gil_Alon/data/raw

Dataset structure:

EXAMPLE DATA
Dataset not found!


## 4. Build Evaluation Index

(Roboflow already split the data - no need to re-split!)

In [ ]:
!python scripts/build_evaluation_index.py

HYPERPARAMETER TUNING SPLIT
Ratios: 70% train / 10% valid / 20% test
Random seed: 42

1. Finding dataset...
Traceback (most recent call last):
  File "/content/Deep_Learning_Project_Gil_Alon/scripts/split_dataset.py", line 246, in <module>
    main()
  File "/content/Deep_Learning_Project_Gil_Alon/scripts/split_dataset.py", line 182, in main
    dataset_dir = find_dataset_dir()
                  ^^^^^^^^^^^^^^^^^^
  File "/content/Deep_Learning_Project_Gil_Alon/scripts/split_dataset.py", line 56, in find_dataset_dir
    raise FileNotFoundError(
FileNotFoundError: Dataset not found in /content/Deep_Learning_Project_Gil_Alon/data/raw. Run download_dataset.py first.


## 6. Verify Results

In [7]:
# Check directory structure
!echo "=== Data Structure ==="
!ls -la data/processed/

!echo ""
!echo "=== Train/Valid/Test Counts ==="
!echo "Train images: $(ls data/processed/train/images/ | wc -l)"
!echo "Valid images: $(ls data/processed/valid/images/ | wc -l)"
!echo "Test images: $(ls data/processed/test/images/ | wc -l)"

=== Data Structure ===
total 8
drwxr-xr-x 2 root root 4096 Jan  8 13:17 .
drwxr-xr-x 4 root root 4096 Jan  8 13:17 ..
-rw-r--r-- 1 root root    0 Jan  8 13:17 .gitkeep

=== Train/Valid/Test Counts ===
ls: cannot access 'data/processed/train/images/': No such file or directory
Train images: 0
ls: cannot access 'data/processed/valid/images/': No such file or directory
Valid images: 0
ls: cannot access 'data/processed/test/images/': No such file or directory
Test images: 0


In [8]:
# Check the evaluation index
import json

with open('data/processed/test_index.json', 'r') as f:
    index = json.load(f)

print("=== Evaluation Index Summary ===")
print(f"Total test images: {index['metadata']['num_images']}")
print(f"Total objects: {index['metadata']['total_objects']}")
print(f"Classes: {index['metadata']['num_classes']}")
print(f"\nDifficulty distribution:")
for diff, count in index['metadata']['difficulty_distribution'].items():
    print(f"  {diff}: {count}")

FileNotFoundError: [Errno 2] No such file or directory: 'data/processed/test_index.json'

In [ ]:
# Show a sample image entry
print("=== Sample Image Entry ===")
sample = index['images'][0]
print(json.dumps(sample, indent=2))

=== Sample Image Entry ===


NameError: name 'index' is not defined

## ✅ Data Pipeline Complete!

Next steps:
- Train models (Step 3)
- Evaluate and compare (Step 4)